In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np

In [2]:
df=pd.read_csv('train.csv')

df['Vehicle_Age'] = df['Vehicle_Age'].astype('category')
df['Vehicle_Age'] = df['Vehicle_Age'].cat.rename_categories({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2})
df['Vehicle_Age'] = df['Vehicle_Age'].astype('int8')

df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category')
df['Vehicle_Damage'] = df['Vehicle_Damage'].cat.rename_categories({'No': 0,'Yes': 1,})
df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('int8')

df['Gender'] = df['Gender'].astype('category')
df['Gender'] = df['Gender'].cat.rename_categories({'Female': 0,'Male': 1,})
df['Gender'] = df['Gender'].astype('int8')

df['Age'] = df['Age'].astype('int8')

df['Driving_License'] = df['Driving_License'].astype('int8')

df['Region_Code'] = df['Region_Code'].astype('int32')

df['Previously_Insured'] = df['Previously_Insured'].astype('int8')

df['Vintage'] = df['Vintage'].astype('int32')

df['Response'] = df['Response'].astype('int8')

df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('int32')

df['Annual_Premium'] = df['Annual_Premium'].astype('int32')

print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype
---  ------                -----
 0   id                    int64
 1   Gender                int8 
 2   Age                   int8 
 3   Driving_License       int8 
 4   Region_Code           int32
 5   Previously_Insured    int8 
 6   Vehicle_Age           int8 
 7   Vehicle_Damage        int8 
 8   Annual_Premium        int32
 9   Policy_Sales_Channel  int32
 10  Vintage               int32
 11  Response              int8 
dtypes: int32(4), int64(1), int8(7)
memory usage: 340.1 MB
None


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,1,21,1,35,0,1,1,65101,124,187,0
1,1,1,43,1,28,0,2,1,58911,26,288,1
2,2,0,25,1,14,1,0,0,38043,152,254,0
3,3,0,35,1,1,0,1,1,2630,156,76,0
4,4,0,36,1,15,1,1,0,31951,152,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,1,48,1,6,0,1,1,27412,26,218,0
11504794,11504794,0,26,1,36,0,0,1,29509,152,115,1
11504795,11504795,0,29,1,32,1,0,0,2630,152,189,0
11504796,11504796,0,51,1,28,0,1,1,48443,26,274,1


In [3]:
df_0=df[df['Response']==0]
df_0

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,1,21,1,35,0,1,1,65101,124,187,0
2,2,0,25,1,14,1,0,0,38043,152,254,0
3,3,0,35,1,1,0,1,1,2630,156,76,0
4,4,0,36,1,15,1,1,0,31951,152,294,0
5,5,0,31,1,47,1,0,0,28150,152,197,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504791,11504791,0,31,1,32,1,0,0,2630,152,204,0
11504792,11504792,1,34,1,28,1,1,0,29974,154,201,0
11504793,11504793,1,48,1,6,0,1,1,27412,26,218,0
11504795,11504795,0,29,1,32,1,0,0,2630,152,189,0


In [4]:
df_1=df[df['Response']==1]
df_1

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1,1,1,43,1,28,0,2,1,58911,26,288,1
7,7,0,47,1,8,0,1,1,40659,26,262,1
28,28,1,40,1,35,0,1,1,2630,157,204,1
36,36,0,50,1,46,0,1,1,29248,124,273,1
44,44,1,40,1,8,0,1,1,36217,124,286,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11504732,11504732,1,45,1,28,0,1,1,32540,124,90,1
11504755,11504755,1,45,1,36,0,1,1,37661,30,163,1
11504784,11504784,1,25,1,28,0,0,1,48240,153,37,1
11504794,11504794,0,26,1,36,0,0,1,29509,152,115,1


In [5]:
weight = len(df_1)/(len(df_1)+len(df_0))
weight

0.12299729208631043

In [6]:
weight_list = np.concatenate((np.ones(len(df_1))*(1-weight),np.ones(len(df_0))*(weight)))
weight_list

array([0.87700271, 0.87700271, 0.87700271, ..., 0.12299729, 0.12299729,
       0.12299729])

In [7]:
df_concat = pd.concat((df_1,df_0))
df_concat

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1,1,1,43,1,28,0,2,1,58911,26,288,1
7,7,0,47,1,8,0,1,1,40659,26,262,1
28,28,1,40,1,35,0,1,1,2630,157,204,1
36,36,0,50,1,46,0,1,1,29248,124,273,1
44,44,1,40,1,8,0,1,1,36217,124,286,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11504791,11504791,0,31,1,32,1,0,0,2630,152,204,0
11504792,11504792,1,34,1,28,1,1,0,29974,154,201,0
11504793,11504793,1,48,1,6,0,1,1,27412,26,218,0
11504795,11504795,0,29,1,32,1,0,0,2630,152,189,0


In [8]:
def age_category(Age):
    if Age > 38:
        return 1
    else:
        return 0

df_concat['Age Above 38'] = df_concat['Age'].apply(age_category)
df_concat['Not_Insured and Damaged'] = df_concat['Vehicle_Damage'] * (1-df_concat['Previously_Insured'])
df_concat

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Age Above 38,Not_Insured and Damaged
1,1,1,43,1,28,0,2,1,58911,26,288,1,1,1
7,7,0,47,1,8,0,1,1,40659,26,262,1,1,1
28,28,1,40,1,35,0,1,1,2630,157,204,1,1,1
36,36,0,50,1,46,0,1,1,29248,124,273,1,1,1
44,44,1,40,1,8,0,1,1,36217,124,286,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504791,11504791,0,31,1,32,1,0,0,2630,152,204,0,0,0
11504792,11504792,1,34,1,28,1,1,0,29974,154,201,0,0,0
11504793,11504793,1,48,1,6,0,1,1,27412,26,218,0,1,1
11504795,11504795,0,29,1,32,1,0,0,2630,152,189,0,0,0


In [9]:
y = df_concat['Response']

x = df_concat.drop(columns=['Response','id'])

#clf = HistGradientBoostingClassifier().fit(x, y,sample_weight=weight_list)
clf = HistGradientBoostingClassifier().fit(x, y)

clf

HistGradientBoostingClassifier()

In [10]:
df=pd.read_csv('test.csv')

df['Vehicle_Age'] = df['Vehicle_Age'].astype('category')
df['Vehicle_Age'] = df['Vehicle_Age'].cat.rename_categories({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2})
df['Vehicle_Age'] = df['Vehicle_Age'].astype('int8')

df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category')
df['Vehicle_Damage'] = df['Vehicle_Damage'].cat.rename_categories({'No': 0,'Yes': 1,})
df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('int8')

df['Gender'] = df['Gender'].astype('category')
df['Gender'] = df['Gender'].cat.rename_categories({'Female': 0,'Male': 1,})
df['Gender'] = df['Gender'].astype('int8')

df['Age'] = df['Age'].astype('int8')

df['Driving_License'] = df['Driving_License'].astype('int8')

df['Region_Code'] = df['Region_Code'].astype('int32')

df['Previously_Insured'] = df['Previously_Insured'].astype('int8')

df['Vintage'] = df['Vintage'].astype('int32')

df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('int32')

df['Annual_Premium'] = df['Annual_Premium'].astype('int32')

print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7669866 entries, 0 to 7669865
Data columns (total 11 columns):
 #   Column                Dtype
---  ------                -----
 0   id                    int64
 1   Gender                int8 
 2   Age                   int8 
 3   Driving_License       int8 
 4   Region_Code           int32
 5   Previously_Insured    int8 
 6   Vehicle_Age           int8 
 7   Vehicle_Damage        int8 
 8   Annual_Premium        int32
 9   Policy_Sales_Channel  int32
 10  Vintage               int32
dtypes: int32(4), int64(1), int8(6)
memory usage: 219.4 MB
None


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,0,20,1,47,0,0,0,2630,160,228
1,11504799,1,47,1,28,0,1,1,37483,124,123
2,11504800,1,47,1,43,0,1,1,2630,26,271
3,11504801,0,22,1,47,1,0,0,24502,152,115
4,11504802,1,51,1,19,0,1,0,34115,124,148
...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,1,57,1,28,0,1,1,51661,124,109
7669862,19174660,1,28,1,50,1,0,0,25651,152,184
7669863,19174661,1,47,1,33,1,1,0,2630,138,63
7669864,19174662,1,30,1,28,0,0,1,38866,124,119


In [11]:
def age_category(Age):
    if Age > 38:
        return 1
    else:
        return 0

df['Age Above 38'] = df['Age'].apply(age_category)
df['Not_Insured and Damaged'] = df['Vehicle_Damage'] * (1-df['Previously_Insured'])
df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Age Above 38,Not_Insured and Damaged
0,11504798,0,20,1,47,0,0,0,2630,160,228,0,0
1,11504799,1,47,1,28,0,1,1,37483,124,123,1,1
2,11504800,1,47,1,43,0,1,1,2630,26,271,1,1
3,11504801,0,22,1,47,1,0,0,24502,152,115,0,0
4,11504802,1,51,1,19,0,1,0,34115,124,148,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,1,57,1,28,0,1,1,51661,124,109,1,1
7669862,19174660,1,28,1,50,1,0,0,25651,152,184,0,0
7669863,19174661,1,47,1,33,1,1,0,2630,138,63,1,0
7669864,19174662,1,30,1,28,0,0,1,38866,124,119,0,1


In [12]:
x = df.drop(columns=['id'])
x

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Age Above 38,Not_Insured and Damaged
0,0,20,1,47,0,0,0,2630,160,228,0,0
1,1,47,1,28,0,1,1,37483,124,123,1,1
2,1,47,1,43,0,1,1,2630,26,271,1,1
3,0,22,1,47,1,0,0,24502,152,115,0,0
4,1,51,1,19,0,1,0,34115,124,148,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,1,57,1,28,0,1,1,51661,124,109,1,1
7669862,1,28,1,50,1,0,0,25651,152,184,0,0
7669863,1,47,1,33,1,1,0,2630,138,63,1,0
7669864,1,30,1,28,0,0,1,38866,124,119,0,1


In [13]:
pred= clf.predict(x)

In [14]:
result = pd.DataFrame(df['id'])
result['Response']=pred
result

,id,Response
0,11504798,0
1,11504799,0
2,11504800,0
3,11504801,0
4,11504802,0
...,...,...
7669861,19174659,0
7669862,19174660,0
7669863,19174661,0
7669864,19174662,1


In [15]:
result.to_csv('result.csv',index=False)